# Lec_15_LeNet-5
<font size=5><b><b></font>
<div align='right'> Hoe Sung Ryu ( 류 회 성 ) </div>
    
<img src= https://miro.medium.com/max/4348/1*PXworfAP2IombUzBsDMg7Q.png width=70%>

    
---
    
Syllabus

|Event Type|Date|Topic|
|--:|:---:|:---|
|1 |July 27| Environment setting and Python basic|
|2 |July 28| Pytorch basic and Custom Data load |
|3 |July 29| Traditional Machine Learning(1) |
|4 |July 30| Traditional Machine Learning(2) |
|5 |July 31| CNN(Convolutional Neural Network)(1)  |
|6 |Aug 03| CNN(Convolutional NeuralNetwork)(2) |
|7 |Aug 04|  RNN(Recurrent Neural Networks)(1) |
|8 |Aug 05|  RNN(Recurrent Neural Networks)(2) |
|9 |Aug 06|  Transfer learning(VGG pertained on ImageNEt for CIfar-10)| 
|10|Aug 07|**Mini_Kaggle**: Facial Expression Recognition on `AffectNet` | 
|11|Aug 08|`Awards` and `Closing`| 

---





<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LeNet-5" data-toc-modified-id="LeNet-5-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>LeNet-5</a></span><ul class="toc-item"><li><span><a href="#References" data-toc-modified-id="References-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>References</a></span></li><li><span><a href="#Setting" data-toc-modified-id="Setting-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Setting</a></span></li><li><span><a href="#Dataset-and-DataLoader" data-toc-modified-id="Dataset-and-DataLoader-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Dataset and DataLoader</a></span></li><li><span><a href="#Model-build" data-toc-modified-id="Model-build-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Model build</a></span></li><li><span><a href="#Model-Compile" data-toc-modified-id="Model-Compile-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Model Compile</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li></ul></div>

## LeNet-5
The basic architecture is shown in the figure below:
<img src= https://miro.medium.com/max/4348/1*PXworfAP2IombUzBsDMg7Q.png width=70%>

LeNet-5 is commonly regarded as the pioneer of convolutional neural networks, consisting of a very simple architecture (by modern standards). In total, LeNet-5 consists of only 7 layers. 3 out of these 7 layers are convolutional layers (C1, C3, C5), which are connected by two average pooling layers (S2 & S4). The penultimate layer is a fully connexted layer (F6), which is followed by the final output layer. The additional details are summarized below:

- All convolutional layers use 5x5 kernels with stride 1.
- The two average pooling (subsampling) layers are 2x2 pixels wide with stride 1.
- Throughrout the network, tanh sigmoid activation functions are used. (**In this notebook, we replace these with ReLU activations**)
- The output layer uses 10 custom Euclidean Radial Basis Function neurons for the output layer. (**In this notebook, we replace these with softmax activations**)
- The input size is 32x32; here, we rescale the MNIST images from 28x28 to 32x32 to match this input dimension. Alternatively, we would have to change the 
achieve error rate below 1% on the MNIST data set, which was very close to the state of the art at the time (produced by a boosted ensemble of three LeNet-4 networks).


### References

- [1] Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proceedings of the IEEE, november 1998.

<div class="alert alert-success" data-title="">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> LeNet-5
  </h1>
</div>

In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

### Setting

In [3]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 32*32
NUM_CLASSES = 10

# Other
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GRAYSCALE = True

### Dataset and DataLoader

In [ ]:
# resize
resize_transform = transforms.Compose([transforms.Resize((32, 32)),
                                       transforms.ToTensor()])

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=resize_transform,
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=resize_transform)


train_loader = DataLoader(##### TODO ##### 
                          batch_size=BATCH_SIZE, 
                          shuffle=##### TODO ##### )

test_loader = DataLoader(##### TODO ##### 
                         batch_size=BATCH_SIZE, 
                         shuffle=##### TODO ##### )

In [5]:
# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 32, 32])
Image label dimensions: torch.Size([128])


### Model build 

```
Fill the blank with the below options:
===========================
a. nn.Conv2d(in_channels, 6, kernel_size=5)
b. nn.Conv2d(6, 16, kernel_size=5)
c. nn.Linear(120, 84)
d. nn.Linear(84, num_classes)
e. nn.Linear(120, 84)
f. nn.Tanh()
g. nn.MaxPool2d(kernel_size=2)
==========================

```

In [6]:
class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = nn.Sequential(
            ### blank_1: conv###
            ### blank_2: tanh ###
            ### blank_3: Pooling ###
            ### blank_4: tanh ###
            ### blank_5: Pooling ###
     
        )    
            
        self.classifier = nn.Sequential(
            ### blank_6: linear###,
            ### blank_7: tanh ###
            ### blank_8: linear ###
            ### blank_9: tanh ###
            ### blank_10: linear ###

        )



    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

### Model Compile

In [7]:
model = LeNet5(NUM_CLASSES, GRAYSCALE)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

### Training 


In [8]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

        

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 0000/0469 | Cost: 2.2918
Epoch: 001/010 | Batch 0050/0469 | Cost: 0.5645
Epoch: 001/010 | Batch 0100/0469 | Cost: 0.3723
Epoch: 001/010 | Batch 0150/0469 | Cost: 0.2367
Epoch: 001/010 | Batch 0200/0469 | Cost: 0.2549
Epoch: 001/010 | Batch 0250/0469 | Cost: 0.1721
Epoch: 001/010 | Batch 0300/0469 | Cost: 0.2147
Epoch: 001/010 | Batch 0350/0469 | Cost: 0.0498
Epoch: 001/010 | Batch 0400/0469 | Cost: 0.1270
Epoch: 001/010 | Batch 0450/0469 | Cost: 0.1093


KeyboardInterrupt: 

### Evaluation 


In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))